In [7]:
import gdal
from osgeo import osr
import sys
import netCDF4
import numpy as np
import xarray as xr
import os
from itertools import groupby

In [2]:
filenames = list(filter(lambda x: x.endswith('.nc'), os.listdir('.')))
filenames = filenames[0:3]
filenames

['pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1991.nc',
 'pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1992.nc',
 'pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1993.nc']

In [31]:
# Averages over an axis. Can group by by season
for filename in filenames:
    file = xr.open_dataset(filename)
    tag = filename.split('.')[0]
    print('Processing ' + tag)
    out = file.mean(dim='time')
    print(out)

Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1991
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.375 1.625 1.875 ...
Data variables:
    pr       (lat, lon) float64 1.119e-06 1.065e-06 1.012e-06 9.582e-07 ...
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1992
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.375 1.625 1.875 ...
Data variables:
    pr       (lat, lon) float64 5.286e-07 4.666e-07 4.045e-07 3.424e-07 ...
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1993
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1

In [15]:
# Rolling operations
def consecWD(arr):
    cwd_count = 0
    max_count = 0
    for val in arr:
        if val >= 0.0000029398:
            cwd_count = cwd_count + 1
            if cwd_count > max_count:
                max_count = cwd_count
        else:
            cwd_count = 0
    return max_count
    
for filename in filenames:
    file = xr.open_dataset(filename)
    tag = filename.split('.')[0]
    print('Processing ' + tag)
    a = file.rolling(time = 3)
    print(a.mean())


Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1991
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 365)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01T12:00:00 1991-01-02T12:00:00 ...
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.375 1.625 1.875 ...
Data variables:
    pr       (time, lat, lon) float64 nan nan nan nan nan nan nan nan nan ...
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1992
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 366)
Coordinates:
  * time     (time) datetime64[ns] 1992-01-01T12:00:00 1992-01-02T12:00:00 ...
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.375 1.625 1.875 ...
Data variables:
    pr       (time, lat, lon) float64 nan nan nan nan nan nan nan nan nan ...
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1993
<xarray.Dataset>
D

In [25]:
# NP reductions
for filename in filenames:
    file = xr.open_dataset(filename)
    tag = filename.split('.')[0]
    print('Processing ' + tag)
    a = file.reduce(np.amax, dim='time') # Only numpy ufuncs
    print(a)

Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1991
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.375 1.625 1.875 ...
Data variables:
    pr       (lat, lon) float64 7.833e-05 7.894e-05 7.954e-05 8.268e-05 ...
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1992
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.375 1.625 1.875 ...
Data variables:
    pr       (lat, lon) float64 6.32e-05 5.417e-05 4.514e-05 3.612e-05 ...
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1993
<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.875 -89.625 -89.375 -89.125 -88.875 -88.625 ...
  * lon      (lon) float32 0.125 0.375 0.625 0.875 1.125 1.

In [34]:
# Arbitrary reductions - more expensive
def pr2in(pr_arr):
    return (pr_arr > 0.0005879).sum()

for filename in filenames:
    file = xr.open_dataset(filename)
    tag = filename.split('.')[0]
    print('Processing ' + tag)
    a = file.to_array()
    print(a.shape)
    
    b = np.apply_along_axis(pr2in, 1, a)
    print(b)

Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1991
(1, 365, 720, 1440)
[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1992
(1, 366, 720, 1440)
[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]
Processing pr_day_BCSD_historical_r1i1p1_ACCESS1-0_1993
(1, 365, 720, 1440)
[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]
